In [1]:
import sys
from pathlib import Path
import numpy as np

# Add baseline directory to path to import helper functions
sys.path.append('baseline')
from baseline.extract_metric import extract_metric_values, extract_multiple_metrics
from utils.run_cmd import run_command, relative
from utils.mrtrix import create_mask, create_response, create_fod, create_peaks, create_white_matter_mask
from utils.tractseg import run_tractseg
from utils.scilpy import run_scilpy_dti, estimate_frf, extract_fodf_metrics
from utils.dwi_data import get_subjects, find_dwi_files
from utils.fslstats import bundle_metrics
from utils.fodf import compute_fodf

/Users/maahes/Documents/MAI/UB/DLMIA/BeyondFA/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration variables - using Path objects
# Path to ds003416 prequal derivatives (resolve to absolute path)
dataset_root = Path("ds003416/derivatives/prequal-v1.0.0").resolve()
output_dir = Path("output").resolve()
tmp_dir = Path("tmp").resolve()  # Use local tmp directory instead of /tmp

# Output directory for intermediate files
tractseg_output_dir = tmp_dir / "tractseg_fa_output"

# Optionally filter specific subjects/sessions (set to None to process all)
subjects_to_process = None  # Process all subjects
# subjects_to_process = ["sub-cIIIs01"]

In [3]:
print("Running BeyondFA baseline on ds003416 prequal derivatives...")

subjects = get_subjects(dataset_root, output_dir, tmp_dir, subjects_to_process)
dwi_files = find_dwi_files(subjects, dataset_root)
print(dwi_files[0])

Running BeyondFA baseline on ds003416 prequal derivatives...

Dataset root: ds003416/derivatives/prequal-v1.0.0
Found 97 subjects
Output directory: output
Temporary directory: tmp
Found 1134 DWI file(s)
Verified 1134 existing DWI file(s)
  - ds003416/.git/annex/objects/W3/Px/MD5E-s163573851--082d3a6c8a32dd850497d23929a005b1.nii.gz/MD5E-s163573851--082d3a6c8a32dd850497d23929a005b1.nii.gz
  - ds003416/.git/annex/objects/J1/2g/MD5E-s10013617--ca95bc9e500c771c65cd0f12cead65f1.nii.gz/MD5E-s10013617--ca95bc9e500c771c65cd0f12cead65f1.nii.gz
  - ds003416/.git/annex/objects/v9/P2/MD5E-s118401266--075e7b8b8bdd0e8c52f485ec6b3c6244.nii.gz/MD5E-s118401266--075e7b8b8bdd0e8c52f485ec6b3c6244.nii.gz
  - ds003416/.git/annex/objects/2q/zv/MD5E-s10092523--3a8691d8f94b43cd4dd533186f20ccfa.nii.gz/MD5E-s10092523--3a8691d8f94b43cd4dd533186f20ccfa.nii.gz
  - ds003416/.git/annex/objects/Mg/PF/MD5E-s119373412--450f63fea7f108c7a66f9d8371283aa9.nii.gz/MD5E-s119373412--450f63fea7f108c7a66f9d8371283aa9.nii.gz
  ... 

In [ ]:
# Process each DWI file
for dwi_nifti_file in dwi_files:
    print(f"\n{'='*60}")
    print(f"Processing: {dwi_nifti_file.stem}")
    print(f"{'='*60}")
    
    # Find corresponding bval and bvec files (BIDS naming convention)
    # They should have the same base name as the nifti file
    base_name = dwi_nifti_file.stem.replace('.nii', '')  # Remove .nii from .nii.gz
    bval_path = dwi_nifti_file.parent / f"{relative(base_name)}.bval"
    bvec_path = dwi_nifti_file.parent / f"{relative(base_name)}.bvec"
    
    # Verify bval and bvec files exist
    if not bval_path.exists():
        print(f"Warning: bval file not found: {relative(bval_path)}, skipping...")
        continue
    if not bvec_path.exists():
        print(f"Warning: bvec file not found: {relative(bvec_path)}, skipping...")
        continue
    
    print(f"Using bval: {relative(bval_path)}")
    print(f"Using bvec: {relative(bvec_path)}")
    
    # Create a unique identifier for this file (subject_session_run)
    # Extract from path: sub-*/ses-*/dwi/*_dwi.nii.gz
    parts = dwi_nifti_file.parts
    subject = [p for p in parts if p.startswith("sub-")][0]
    session = [p for p in parts if p.startswith("ses-")][0]
    run_info = base_name.replace(f"{subject}_{session}_", "").replace("_dwi", "")
    file_id = f"{subject}_{session}_{run_info}"
    
    # Create directories
    tractseg_dir = tractseg_output_dir / file_id / "tractseg"
    tractseg_dir.mkdir(parents=True, exist_ok=True)
    

    print("Creating mask, response, FODs, and peaks...")
    
    # Create mask
    mask_file = tractseg_dir / "nodif_brain_mask.nii.gz"
    create_mask(dwi_nifti_file, mask_file, bvec_path, bval_path)
    
    # Check number of gradient directions before proceeding with CSD
    # Read bval file to count non-zero b-values (gradient directions)
    with open(bval_path, 'r') as f:
        bvals = [float(x) for x in f.read().strip().split()]
    
    num_gradient_directions = sum(1 for bval in bvals if bval > 0)
    min_directions_for_csd = 6  # CSD requires at least 6 directions for lmax=2
    
    if num_gradient_directions < min_directions_for_csd:
        print(f"  WARNING: Dataset has only {num_gradient_directions} gradient directions")
        print(f"  CSD requires at least {min_directions_for_csd} directions. Skipping CSD/FOD/peaks/TractSeg steps.")
        print(f"  Will only calculate DTI metrics (FA) for this dataset.")
        skip_csd = True
    else:
        print(f"  Dataset has {num_gradient_directions} gradient directions (sufficient for CSD)")
        skip_csd = False
    
    # Create response
    response_file = tractseg_dir / "response.txt"
    create_response(dwi_nifti_file, response_file, bvec_path, bval_path, skip_csd)
    
    # Create FODs
    fod_file = tractseg_dir / "WM_FODs.nii.gz"
    create_fod(dwi_nifti_file, fod_file, response_file, bvec_path, bval_path, skip_csd)
    
    # Create peaks
    peaks_file = tractseg_dir / "peaks.nii.gz"
    create_peaks(fod_file, peaks_file, mask_file, skip_csd)
    
    # Run TractSeg
    # Check if TractSeg output exists (check for bundle_segmentations directory)
    bundle_roi_dir = tractseg_dir / "bundle_segmentations"
    run_tractseg(peaks_file, bundle_roi_dir, tractseg_dir, bval_path, bvec_path, mask_file, skip_csd)

    # Run DTI metrics calculation 
    fa_dir = tractseg_output_dir / file_id / "metric"
    fa_dir.mkdir(parents=True, exist_ok=True)
    run_scilpy_dti(fa_dir, dwi_nifti_file, mask_file, bval_path, bvec_path)
    
    # Get corresponding metrics for all 4 metrics: FA, MD, AD, RD
    print("Calculating average metrics in bundles...")
    # bundle_roi_dir already defined above for caching check
    
    # Make json with mean of metrics in bundle
    if skip_csd or not bundle_roi_dir.exists():
        print(f"  Skipping bundle metrics calculation (TractSeg was skipped due to insufficient directions)")
        roi_list = []
    else:
        roi_list = sorted(bundle_roi_dir.glob("*.nii.gz"))

    # fodf metric pipeline
    fodf_dir = tractseg_output_dir / file_id / "fodf"
    fodf_dir.mkdir(parents=True, exist_ok=True)
    # create white matter mask
    wm_mask_file = fodf_dir / "wm_mask.nii.gz"
    create_white_matter_mask(dwi_nifti_file, wm_mask_file)
    # estimate fiber response function
    frf_file = fodf_dir / "frf.txt"
    estimate_frf(dwi_nifti_file, mask_file, wm_mask_file, frf_file, bval_path, bvec_path)
    # compute fodf
    fodf_file = fodf_dir / "fodf.nii.gz"
    compute_fodf(dwi_nifti_file, bval_path, bvec_path, frf_file, fodf_file, mask_file)
    # compute fodf metrics: AFD_total, NuFO
    extract_fodf_metrics(fa_dir, fodf_file, mask_file)

    # Create tensor metrics files for each metric
    metrics = ['fa', 'md', 'ad', 'rd', 'afd_total', 'nufo']
    tensor_metrics_files = bundle_metrics(fa_dir, roi_list, tractseg_output_dir, file_id, metrics)
    
    # Extract all metrics and combine into 512-element vector
    print(f"Extracting and combining all metrics to {relative(tractseg_output_dir)}...")
    
    # Check if final output already exists
    feature_size = 72 * len(metrics)
    output_name = output_dir / f"{file_id}_features-{feature_size}.json"
    if output_name.exists():
        print(f"  [CACHE] Final output already exists: {relative(output_name)}, skipping extraction")
    else:
        # Prepare metric files dictionary
        metric_files_dict = {}
        for metric in metrics:
            if tensor_metrics_files[metric].exists():
                metric_files_dict[metric] = str(tensor_metrics_files[metric])
        
        # Combine all metrics into a vector
        combined_json_file = tractseg_output_dir / file_id / "combined_metrics.json"
        extract_multiple_metrics(metric_files_dict, str(combined_json_file), metrics)
        
        # Save the final metric.json to output directory with unique name
        print(f"All metrics saved to {relative(output_name)}!")
        output_dir.mkdir(parents=True, exist_ok=True)
        combined_json_file.replace(output_name)
    
    print(f"Processing complete for {file_id}!")

print(f"All completed!")


Processing: sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215_dwi.nii
Using bval: ds003416/derivatives/prequal-v1.0.0/sub-cIVs082/ses-s1Bx2/dwi/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215_dwi.bval
Using bvec: ds003416/derivatives/prequal-v1.0.0/sub-cIVs082/ses-s1Bx2/dwi/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215_dwi.bvec
Creating mask, response, FODs, and peaks...
  [CACHE] Mask already exists: tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/tractseg/nodif_brain_mask.nii.gz, skipping dwi2mask
  Dataset has 56 gradient directions (sufficient for CSD)
  [CACHE] Response already exists: tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/tractseg/response.txt, skipping dwi2response
  [CACHE] FOD already exists: tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/tractseg/WM_FODs.nii.gz, skipping dwi2fod
  [CACHE] Peaks already exists: tmp/tractseg_fa_output/sub-cIVs08

  mrthreshold: [WARNING] existing output files will be overwritten
  mrthreshold: uncompressing image "/Users/maahes/Documents/MAI/UB/DLMIA/BeyondFA/ds003416/derivatives/prequal-v1.0.0/sub-cIVs082/ses-s1Bx2/dwi/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215_dwi.nii.gz"... [==================================================]
  mrthreshold: [WARNING] requested datatype (Bit) not supported - substituting with UInt8
  mrthreshold: Determining and applying per-volume thresholds... [==================================================]
  mrthreshold: compressing image "/var/folders/5z/rgx8sdx575v6n3c32wj0dj5c0000gn/T/tmp_md0g48f.nii.gz"... [==================================================]
  mrconvert: [WARNING] existing output files will be overwritten
  mrconvert: uncompressing image "/var/folders/5z/rgx8sdx575v6n3c32wj0dj5c0000gn/T/tmp_md0g48f.nii.gz"... [==================================================]
  mrconvert: copying from "/var/folde...0000gn/T/tmp_md0g48f.nii.gz" to "

Running: mrconvert /var/folders/5z/rgx8sdx575v6n3c32wj0dj5c0000gn/T/tmp_md0g48f.nii.gz /Users/maahes/Documents/MAI/UB/DLMIA/BeyondFA/tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/fodf/wm_mask.nii.gz -datatype uint8 -force


  mrconvert: compressing image "/Users/maahes/Documents/MAI/UB/DLMIA/BeyondFA/tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/fodf/wm_mask.nii.gz"... [==================================================]
  mrthreshold: [WARNING] existing output files will be overwritten
  mrthreshold: uncompressing image "/Users/maahes/Documents/MAI/UB/DLMIA/BeyondFA/ds003416/derivatives/prequal-v1.0.0/sub-cIVs082/ses-s1Bx2/dwi/sub-cIVs082_ses-s1Bx2_acq-b1000n3r21x21x22peAPA_run-216_dwi.nii.gz"... [==================================================]
  mrthreshold: [WARNING] requested datatype (Bit) not supported - substituting with UInt8
  mrthreshold: Determining and applying per-volume thresholds... [==================================================]
  mrthreshold: compressing image "/var/folders/5z/rgx8sdx575v6n3c32wj0dj5c0000gn/T/tmpsm21g4cs.nii.gz"... [==================================================]
  mrconvert: [WARNING] existing output files will be overwritte

  [CACHE] Fiber response function already exists: tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/fodf/frf.txt, skipping estimate_frf
  [CACHE] fODF already exists: tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/fodf/fodf.nii.gz, skipping computation
  [CACHE] FODF metrics already exist: tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/metric/afd_total.nii.gz, tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/metric/nufo.nii.gz, skipping extract_fodf_metrics
  [CACHE] Tensor metrics files already exist, skipping fslstats calculations
Extracting and combining all metrics to tmp/tractseg_fa_output...
Extracted 432 values (combined metrics) saved to tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215/combined_metrics.json
All metrics saved to output/sub-cIVs082_ses-s1Bx2_acq-b2000n56r21x21x22peAPP_run-215_features-None.json!
Processing c

...

  mrconvert: compressing image "/Users/maahes/Documents/MAI/UB/DLMIA/BeyondFA/tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b1000n40r21x21x22peAPP_run-114/fodf/wm_mask.nii.gz"... [==================================================]


  [CACHE] Fiber response function already exists: tmp/tractseg_fa_output/sub-cIVs082_ses-s1Bx2_acq-b1000n40r21x21x22peAPP_run-114/fodf/frf.txt, skipping estimate_frf
Computing fODF from ds003416/.git/annex/objects/Mg/PF/MD5E-s119373412--450f63fea7f108c7a66f9d8371283aa9.nii.gz/MD5E-s119373412--450f63fea7f108c7a66f9d8371283aa9.nii.gz...


/Users/maahes/Documents/MAI/UB/DLMIA/BeyondFA/.venv/lib/python3.12/site-packages/dipy/testing/decorators.py:192: UserWarning: Number of parameters required for the fit are more than the actual data points
  return func(*args, **kwargs)


  Fitting CSD model...


KeyboardInterrupt: 